# Introduction/Business Problem

### Ankara is the capital city of Turkey. It includes lots of universities and work places. Students and workers are the important customers of  restaurants. For this reason they search good places to eat or drink something. I will be work on recommending a place to stakeholders who wants open a restaurant in Ankara which is capital of Turkey. The project will be useful for entrepreneur who will open a new restaurant and someone who searchs a good restaurant in Ankara.


# Data
 ### I will use data which is accessible with using Foursquare API. I am taking rating of each restaurant venue and i will sort results according to rating values. Places that have high rating values will be recommend to the users.

In [89]:
import pandas as pd
import numpy as np
import requests
from bs4 import *

In [90]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [91]:
CLIENT_ID = 'HRLVLMF22OHM5APAVNZCUPO2SP2LBDUMIEQXROB0MMWLJCPP' # your Foursquare ID
CLIENT_SECRET = 'FSFUSGKFLMFF2ZBKWTFU1TWHRDNBABHAFXQUL00IXFGKQ0WB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
ACCESS_TOKEN = 'C32X24TRBRYRWIXHFDN25CVLFZ2ZOUCRJWWLT3FZZZH43HJA' # your FourSquare Access Token

In [93]:
address = 'Ankara'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Ankara are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Ankara are 39.9207893, 32.8540412.


In [7]:
import folium

ModuleNotFoundError: No module named 'folium'

In [94]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=HRLVLMF22OHM5APAVNZCUPO2SP2LBDUMIEQXROB0MMWLJCPP&client_secret=FSFUSGKFLMFF2ZBKWTFU1TWHRDNBABHAFXQUL00IXFGKQ0WB&v=20180605&ll=39.9207893,32.8540412&radius=500&limit=100'

In [95]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607943bff98e863be71113b8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Kızılay',
  'headerFullLocation': 'Kızılay, Ankara',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 203,
  'suggestedBounds': {'ne': {'lat': 39.925289304500005,
    'lng': 32.8598977823151},
   'sw': {'lat': 39.9162892955, 'lng': 32.84818461768489}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ba889e6f964a5201bdf39e3',
       'name': 'Dost Kitabevi',
       'location': {'address': 'Karanfil Sk. No:11',
        'crossStreet': 'Yüksel Caddesi',
        'lat': 39.919727056859394,
        'lng': 32.855377270228324,
        'labeledLatLngs': [{'label': 

In [96]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [97]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
print(nearby_venues)
# filter the category for each row
nearby_venues['venue.categories'] =nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

                                       venue.name  \
0                                   Dost Kitabevi   
1                                     Tripod Foto   
2                            Yapı Kredi Yayınları   
3                                         GSStore   
4                        Kalem Kitap Kültür Sanat   
..                                            ...   
95                                         Subway   
96                              Sefer Vitamin Bar   
97                           Muhabbet Cafe&Bistro   
98                               Yenişehir Hamamı   
99  Korkmaz Simsek Spor Merkezi - Wing Tsun Okulu   

                                     venue.categories  venue.location.lat  \
0   [{'id': '4bf58dd8d48988d114951735', 'name': 'B...           39.919727   
1   [{'id': '4eb1bdf03b7b55596b4a7491', 'name': 'C...           39.920695   
2   [{'id': '4bf58dd8d48988d114951735', 'name': 'B...           39.921322   
3   [{'id': '4bf58dd8d48988d1f2941735', 'name': 'S...  

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Dost Kitabevi,Bookstore,39.919727,32.855377
1,Tripod Foto,Camera Store,39.920695,32.851861
2,Yapı Kredi Yayınları,Bookstore,39.921322,32.854022
3,GSStore,Sporting Goods Shop,39.921364,32.853649
4,Kalem Kitap Kültür Sanat,Bookstore,39.920384,32.851633


In [98]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [99]:
print(nearby_venues['categories'].unique())

['Bookstore' 'Camera Store' 'Sporting Goods Shop' 'Gift Shop' 'Boutique'
 'Kuruyemişçi' 'Café' "Women's Store" 'Pedestrian Plaza' 'Art Gallery'
 'Dessert Shop' 'Gym / Fitness Center' 'Optical Shop' 'Bar'
 'Tourist Information Center' 'Arcade' 'Clothing Store' 'Bakery'
 'Flower Shop' 'Turkish Restaurant' 'Lounge' 'Coffee Shop'
 'Arts & Crafts Store' 'Restaurant' 'Used Bookstore' 'Dive Shop'
 'Seafood Restaurant' 'Tea Room' 'Turkish Home Cooking Restaurant'
 'Theater' 'Paper / Office Supplies Store' 'Pub' 'Plaza' 'Astrologer'
 'Doner Restaurant' 'Fraternity House' 'Smoke Shop' 'Soup Place' 'Diner'
 'Sandwich Place' 'Movie Theater' 'Cigkofte Place' 'Theme Restaurant'
 'Burger Joint' 'Hotel' 'Kebab Restaurant' 'Cosmetics Shop' 'Shoe Store'
 'Beer Garden' 'Snack Place' 'Spa' 'Gym']


In [100]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #res= results["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [101]:
ankara_venues=getNearbyVenues(names=nearby_venues['name'],
                               latitudes=nearby_venues['lat'],
                               longitudes=nearby_venues['lng']
                                  )

Dost Kitabevi
Tripod Foto
Yapı Kredi Yayınları
GSStore
Kalem Kitap Kültür Sanat
Paşabahçe mağazası
Full Black
Tuğba Kuruyemiş
Fiesta Cafe Bistro
addax
Sakarya Caddesi
Ankara Sinema Akademisi
Ali Uzun Geleneksel Şekerci
Fidan Kultur Cafe
Dior Gym
Helvacı Ali
Serdar Optik
Ardıç Kitabevi
Hezarfen Teras Bar
Berlika Tur
Fantasyland Oyun Merkezi & Coffee
Koton
Cafe Down
Filiz Çay Evi
Çiçekçi Mahmut
EY Club
Ümit Usta Ciğerim
Fenerium
Leyla Optik
Yargıtay Lokali
DeFacto
Taraftar Store
İş Bankası Kültür Yayınları
Bolulu Hasan Usta
The Muddy Waters
Starbucks
Starbucks
Mekan Kafe
Galeri Ekin
Ata Aspava
Berdelacuz Sahaf/ Ankara
Yüce Erek Sahaf
Aqua Club Dalgıç Okulu
Kocatepe Kahve Evi
Meşhur Adıyaman Çiğ Köftecisi Samet Usta
Can Balık
Eylül Cafe
Dalkılıç Spor
Deva Kebap ve İşkembecisi
UĞURFOTO
Ankara Sanat Tiyatrosu
SEYRİ KEYF CAFE
Galeri Ekin Kırtasiye
Ahır Pub
Besi Balık Pişirme
Kızılay Meydanı
Ankara Roof
OT
Deniz Erin Astroloji ve Danışmanlık Merkezi
Degaje
Back House Grunge Pub
Kebo
Kafe Kara

KeyError: 'groups'

In [56]:
print(istanbul_venues.shape)
istanbul_venues.head()

(10000, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Dost Kitabevi,39.919727,32.855377,Dost Kitabevi,39.919727,32.855377,Bookstore
1,Dost Kitabevi,39.919727,32.855377,Yapı Kredi Yayınları,39.921322,32.854022,Bookstore
2,Dost Kitabevi,39.919727,32.855377,Fiesta Cafe Bistro,39.921041,32.856682,Café
3,Dost Kitabevi,39.919727,32.855377,Ankara Sinema Akademisi,39.919199,32.855625,Art Gallery
4,Dost Kitabevi,39.919727,32.855377,Ardıç Kitabevi,39.919205,32.855979,Bookstore


In [57]:
nearby_venues

,name,categories,lat,lng
0,Dost Kitabevi,Bookstore,39.919727,32.855377
1,Tripod Foto,Camera Store,39.920695,32.851861
2,Yapı Kredi Yayınları,Bookstore,39.921322,32.854022
3,GSStore,Sporting Goods Shop,39.921364,32.853649
4,Kalem Kitap Kültür Sanat,Bookstore,39.920384,32.851633
...,...,...,...,...
95,Subway,Sandwich Place,39.919879,32.851551
96,Sefer Vitamin Bar,Snack Place,39.922710,32.850341
97,Muhabbet Cafe&Bistro,Café,39.920919,32.852309
98,Yenişehir Hamamı,Spa,39.923224,32.851608


In [58]:
search_query = 'cafe'
radius = 500
print(search_query + ' .... OK!')

cafe .... OK!


In [59]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=HRLVLMF22OHM5APAVNZCUPO2SP2LBDUMIEQXROB0MMWLJCPP&client_secret=FSFUSGKFLMFF2ZBKWTFU1TWHRDNBABHAFXQUL00IXFGKQ0WB&ll=39.9207893,32.8540412&oauth_token=C32X24TRBRYRWIXHFDN25CVLFZ2ZOUCRJWWLT3FZZZH43HJA&v=20180605&query=cafe&radius=500&limit=100'

In [60]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60793e28bf0a0a3f9a6b079b'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4c7556dbb474a1cdacf9b8bf',
    'name': 'Ezgi Cafe',
    'location': {'address': 'Konur Sk. No:14/A Kızılay',
     'lat': 39.91837283124024,
     'lng': 32.85576717739065,
     'labeledLatLngs': [{'label': 'display',
       'lat': 39.91837283124024,
       'lng': 32.85576717739065}],
     'distance': 306,
     'postalCode': '06420',
     'cc': 'TR',
     'neighborhood': 'Çankaya',
     'city': 'Ankara',
     'state': 'Ankara',
     'country': 'Türkiye',
     'formattedAddress': ['Konur Sk. No:14/A Kızılay', '06420 Ankara']},
    'categories': [{'id': '4bf58dd8d48988d16d941735',
      'name': 'Café',
      'pluralName': 'Cafés',
      'shortName': 'Café',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1618558504',

In [61]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,4c7556dbb474a1cdacf9b8bf,Ezgi Cafe,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1618558504,False,Konur Sk. No:14/A Kızılay,39.918373,32.855767,"[{'label': 'display', 'lat': 39.91837283124024...",306,06420,TR,Çankaya,Ankara,Ankara,Türkiye,"[Konur Sk. No:14/A Kızılay, 06420 Ankara]",NaN,NaN
1,583ea7808f0be457d3b9e5a1,Moda Cafe&Bistro,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1618558504,False,"Meşrutiyet, Atatürk Blv No:95, 06420 Kızılay A...",39.920423,32.854619,"[{'label': 'display', 'lat': 39.92042269097689...",64,06420,TR,Çankaya,Ankara,Ankara,Türkiye,"[Meşrutiyet, Atatürk Blv No:95, 06420 Kızılay ...",Atatürk Caddesi,NaN
2,4c6278d4e1621b8dea792353,Flz Cafe & Restaurant,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1618558504,False,GMK Blv. YKM Mağazası 8. Kat Kızılay,39.920764,32.852817,"[{'label': 'display', 'lat': 39.92076443347085...",104,06650,TR,Kızılay,Ankara,Ankara,Türkiye,[GMK Blv. YKM Mağazası 8. Kat Kızılay (Gazi Mu...,Gazi Mustafa Kemal Blv.,NaN
3,4c38b73918e72d7fd5d818f5,Su Cafe,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1618558504,False,Konur Sk. No:6 Kızılay,39.918823,32.856081,"[{'label': 'display', 'lat': 39.91882288025211...",279,06420,TR,Çankaya,Ankara,Ankara,Türkiye,"[Konur Sk. No:6 Kızılay (Meşrutiyet Cad.), 064...",Meşrutiyet Cad.,87032514
4,4ebaa2268b818968846e8bea,Turuncu Cafe Pub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",v-1618558504,False,Meşrutiyet Cd. Konur 1 Sk. No:15/3 Kızılay,39.918450,32.856347,"[{'label': 'display', 'lat': 39.91844982632325...",326,06660,TR,NaN,Ankara,Ankara,Türkiye,[Meşrutiyet Cd. Konur 1 Sk. No:15/3 Kızılay (K...,Karanfil Sokak,1358848112


In [62]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id
0,Ezgi Cafe,Café,Konur Sk. No:14/A Kızılay,39.918373,32.855767,"[{'label': 'display', 'lat': 39.91837283124024...",306,06420,TR,Çankaya,Ankara,Ankara,Türkiye,"[Konur Sk. No:14/A Kızılay, 06420 Ankara]",NaN,4c7556dbb474a1cdacf9b8bf
1,Moda Cafe&Bistro,Café,"Meşrutiyet, Atatürk Blv No:95, 06420 Kızılay A...",39.920423,32.854619,"[{'label': 'display', 'lat': 39.92042269097689...",64,06420,TR,Çankaya,Ankara,Ankara,Türkiye,"[Meşrutiyet, Atatürk Blv No:95, 06420 Kızılay ...",Atatürk Caddesi,583ea7808f0be457d3b9e5a1
2,Flz Cafe & Restaurant,Café,GMK Blv. YKM Mağazası 8. Kat Kızılay,39.920764,32.852817,"[{'label': 'display', 'lat': 39.92076443347085...",104,06650,TR,Kızılay,Ankara,Ankara,Türkiye,[GMK Blv. YKM Mağazası 8. Kat Kızılay (Gazi Mu...,Gazi Mustafa Kemal Blv.,4c6278d4e1621b8dea792353
3,Su Cafe,Café,Konur Sk. No:6 Kızılay,39.918823,32.856081,"[{'label': 'display', 'lat': 39.91882288025211...",279,06420,TR,Çankaya,Ankara,Ankara,Türkiye,"[Konur Sk. No:6 Kızılay (Meşrutiyet Cad.), 064...",Meşrutiyet Cad.,4c38b73918e72d7fd5d818f5
4,Turuncu Cafe Pub,Pub,Meşrutiyet Cd. Konur 1 Sk. No:15/3 Kızılay,39.918450,32.856347,"[{'label': 'display', 'lat': 39.91844982632325...",326,06660,TR,NaN,Ankara,Ankara,Türkiye,[Meşrutiyet Cd. Konur 1 Sk. No:15/3 Kızılay (K...,Karanfil Sokak,4ebaa2268b818968846e8bea
5,Sem Cafe,Café,Konur Sk. No:15/A Kızılay,39.918534,32.856636,"[{'label': 'display', 'lat': 39.91853384862023...",334,NaN,TR,Çankaya,Ankara,Ankara,Türkiye,"[Konur Sk. No:15/A Kızılay (Meşrutiyet Cd.), A...",Meşrutiyet Cd.,4daaba82fa8cc764973ba5d1
6,Cafe Taş Fırın Ormancı,Bakery,Konur Sk. No:12/B Kızılay,39.918503,32.856175,"[{'label': 'display', 'lat': 39.91850332290211...",312,06650,TR,NaN,Kızılay,Ankara,Türkiye,"[Konur Sk. No:12/B Kızılay, 06650 Ankara]",NaN,4d7251fa5838a0932e3b09db
7,Alaros Yüksel Cafe & Bistro,Café,Karanfil Sokak,39.919417,32.855871,"[{'label': 'display', 'lat': 39.91941672429775...",218,NaN,TR,NaN,Ankara,Ankara,Türkiye,"[Karanfil Sokak, Ankara]",NaN,537056ec498e0be35cfe2b65
8,Fiesta Cafe Bistro,Café,Kocatepe Mah. Ziya Gökalp Cad. No:12 Kızılay,39.921041,32.856682,"[{'label': 'display', 'lat': 39.92104102215064...",227,06420,TR,Çankaya,Ankara,Ankara,Türkiye,"[Kocatepe Mah. Ziya Gökalp Cad. No:12 Kızılay,...",NaN,586c03d203e29a7b9204580e
9,Çatı Cafe,Café,kizilay,39.921701,32.854959,"[{'label': 'display', 'lat': 39.92170097419393...",128,NaN,TR,Çankaya,Ankara,Ankara,Türkiye,"[kizilay, Ankara]",NaN,4e16e5abc65be9c55e8933a7


In [87]:
ratings={}
for i in range(len(dataframe_filtered)):
    venue_id=dataframe_filtered['id'][i]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

    result = requests.get(url).json()
    try:
        rating=result['response']['venue']['rating']
        ratings[i]={'id':dataframe_filtered['id'][i],'rating':rating}
    except:
        print('This venue has not been rated yet.')
    

This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not been rated yet.
This venue has not b

In [78]:
for i in range(len(dataframe_filtered)):
    for j in range(len(ratings)):
        if j in ratings:
            if dataframe_filtered['id'][i]==ratings[j]['id']:
                dataframe_filtered['rating'][i]=ratings[j]['rating']
            else:
                continue

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [79]:
dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,neighborhood,city,state,country,formattedAddress,crossStreet,id,rating
0,Ezgi Cafe,Café,Konur Sk. No:14/A Kızılay,39.918373,32.855767,"[{'label': 'display', 'lat': 39.91837283124024...",306,06420,TR,Çankaya,Ankara,Ankara,Türkiye,"[Konur Sk. No:14/A Kızılay, 06420 Ankara]",NaN,4c7556dbb474a1cdacf9b8bf,6.3
1,Moda Cafe&Bistro,Café,"Meşrutiyet, Atatürk Blv No:95, 06420 Kızılay A...",39.920423,32.854619,"[{'label': 'display', 'lat': 39.92042269097689...",64,06420,TR,Çankaya,Ankara,Ankara,Türkiye,"[Meşrutiyet, Atatürk Blv No:95, 06420 Kızılay ...",Atatürk Caddesi,583ea7808f0be457d3b9e5a1,7.6
2,Flz Cafe & Restaurant,Café,GMK Blv. YKM Mağazası 8. Kat Kızılay,39.920764,32.852817,"[{'label': 'display', 'lat': 39.92076443347085...",104,06650,TR,Kızılay,Ankara,Ankara,Türkiye,[GMK Blv. YKM Mağazası 8. Kat Kızılay (Gazi Mu...,Gazi Mustafa Kemal Blv.,4c6278d4e1621b8dea792353,7.2
3,Su Cafe,Café,Konur Sk. No:6 Kızılay,39.918823,32.856081,"[{'label': 'display', 'lat': 39.91882288025211...",279,06420,TR,Çankaya,Ankara,Ankara,Türkiye,"[Konur Sk. No:6 Kızılay (Meşrutiyet Cad.), 064...",Meşrutiyet Cad.,4c38b73918e72d7fd5d818f5,6.1
4,Turuncu Cafe Pub,Pub,Meşrutiyet Cd. Konur 1 Sk. No:15/3 Kızılay,39.918450,32.856347,"[{'label': 'display', 'lat': 39.91844982632325...",326,06660,TR,NaN,Ankara,Ankara,Türkiye,[Meşrutiyet Cd. Konur 1 Sk. No:15/3 Kızılay (K...,Karanfil Sokak,4ebaa2268b818968846e8bea,7.2
5,Sem Cafe,Café,Konur Sk. No:15/A Kızılay,39.918534,32.856636,"[{'label': 'display', 'lat': 39.91853384862023...",334,NaN,TR,Çankaya,Ankara,Ankara,Türkiye,"[Konur Sk. No:15/A Kızılay (Meşrutiyet Cd.), A...",Meşrutiyet Cd.,4daaba82fa8cc764973ba5d1,6.4
6,Cafe Taş Fırın Ormancı,Bakery,Konur Sk. No:12/B Kızılay,39.918503,32.856175,"[{'label': 'display', 'lat': 39.91850332290211...",312,06650,TR,NaN,Kızılay,Ankara,Türkiye,"[Konur Sk. No:12/B Kızılay, 06650 Ankara]",NaN,4d7251fa5838a0932e3b09db,6.3
7,Alaros Yüksel Cafe & Bistro,Café,Karanfil Sokak,39.919417,32.855871,"[{'label': 'display', 'lat': 39.91941672429775...",218,NaN,TR,NaN,Ankara,Ankara,Türkiye,"[Karanfil Sokak, Ankara]",NaN,537056ec498e0be35cfe2b65,6.2
8,Fiesta Cafe Bistro,Café,Kocatepe Mah. Ziya Gökalp Cad. No:12 Kızılay,39.921041,32.856682,"[{'label': 'display', 'lat': 39.92104102215064...",227,06420,TR,Çankaya,Ankara,Ankara,Türkiye,"[Kocatepe Mah. Ziya Gökalp Cad. No:12 Kızılay,...",NaN,586c03d203e29a7b9204580e,8.8
9,Çatı Cafe,Café,kizilay,39.921701,32.854959,"[{'label': 'display', 'lat': 39.92170097419393...",128,NaN,TR,Çankaya,Ankara,Ankara,Türkiye,"[kizilay, Ankara]",NaN,4e16e5abc65be9c55e8933a7,6.3
